In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Prepare dataset for training

data = pd.read_csv("sarcasm_data.csv")

# Split data by gender (F/M)
data_F = data[data['gender'] == 'F']
data_M = data[data['gender'] == 'M']

X_F = data_F['utterance']
y_F = data_F['sarcasm']

X_M = data_M['utterance']
y_M = data_M['sarcasm']

# Split data into training, validation, and test sets
X_F_train, X_F_temp, y_F_train, y_F_temp = train_test_split(X_F, y_F, test_size=0.2, random_state=42)
X_F_val, X_F_test, y_F_val, y_F_test = train_test_split(X_F_temp, y_F_temp, test_size=0.5, random_state=42)

X_M_train, X_M_temp, y_M_train, y_M_temp = train_test_split(X_M, y_M, test_size=0.2, random_state=42)
X_M_val, X_M_test, y_M_val, y_M_test = train_test_split(X_M_temp, y_M_temp, test_size=0.5, random_state=42)

In [ ]:
# Tokenize data

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(X_F_train), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(list(X_F_val), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(list(X_F_test), truncation=True, padding=True, return_tensors='pt')

train_labels = torch.tensor(list(y_F_train))
val_labels = torch.tensor(list(y_F_val))
test_labels = torch.tensor(list(y_F_test))

In [ ]:
# Load pre-trained BERT model

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
# Modify model architecture

In [ ]:
# Fine-tune the model

device = torch.device("cpu")
model.to(device)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss}, Val Acc: {accuracy}')

In [ ]:
# Evaluate the model

test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
accuracy = correct / total
print(f'Test Loss: {test_loss}, Test Acc: {accuracy}')